In [4]:
import scipy.io
import numpy as np
import pyedflib 
import os
import glob
import h5py
import mne
# import edfio
import pandas as pd
from pyedflib import FILETYPE_BDF, FILETYPE_BDFPLUS, FILETYPE_EDF, FILETYPE_EDFPLUS
import GenData

JUST AN EXAMPLE

In [ ]:
# all available WESA data is planned for test split
data_path = 'C:\\Users\\MLaccount\\projects\\Data\\WESA HC\\*.mat'
gt_path = 'C:\\Users\\MLaccount\\projects\\RBD\\SS_ground_truth\\WESA\\*.txt'

data = glob.glob(data_path)
gt = glob.glob(gt_path)
test_eegs, test_emgs, test_eogs, subjects_test = GenData.extract_eeg_emg_eog(data, _, gt, seg_len=30, fs=250, marker = 'WESA')

In [ ]:
# SAVE SIGNALS BACK TO EDF FILE ONLY WITH RELEVANT CHANNELS
for jj in range(len(test_eegs)):
    data = {}
    
    data['EEG'] = np.reshape(test_eegs[jj], (7500*len(test_eegs[jj])))
    data['EMG'] = np.reshape(test_emgs[jj], (7500*len(test_emgs[jj])))
    data['EOG'] = np.reshape(test_eogs[jj], (7500*len(test_eogs[jj])))
    
    data_df = pd.DataFrame({k: pd.Series(v) for k, v in data.items()})
    data_np = np.round(data_df.to_numpy(dtype=np.float64), 4)
    print(np.min(data_np), np.max(data_np))
    # data_np = data_np/1e6
    # print(data_np)
    
    info = mne.create_info(ch_names=['EEG', 'EMG', 'EOG'], sfreq=250, ch_types=['eeg', 'emg', 'eog'])
    
    raw = mne.io.RawArray(data_np.T, info)
    
    n_channels = len(raw.get_data())
    channels = raw.get_data()
    print(np.min(channels), np.max(channels))
    
    file_type = FILETYPE_EDF
    f = pyedflib.EdfWriter(f'{subjects_test[jj]}.edf', n_channels=n_channels, file_type=file_type)
    
    dmin, dmax = -32768, 32767 
    sfreq = raw.info['sfreq']
    channel_info = []
    
    ch_idx = range(n_channels) 
    for i in ch_idx:
        ch_dict = {'label': raw.ch_names[i], 
                   'dimension': 'uV',
                   'sample_rate': sfreq, 
                   'physical_min': channels.min(), 
                   'physical_max': channels.max(), 
                   'digital_min':  dmin, 
                   'digital_max':  dmax, 
                   'transducer': '', 
                   'prefilter': ''}
    
        channel_info.append(ch_dict)
    
    f.setSignalHeaders(channel_info)
    f.writeSamples(channels)
    f.close()